In [1]:
import geopandas as gpd

import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import tensorflow as tf
import sys
import os
import glob

from functools import partial
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt




code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRLinearModel, PerturbedBPRMLPModel
from make_datasets import make_data


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)
from zinf_gp.metrics import normcdf, fixed_top_X



from perturbations import perturbed
from bpr import bpr_variable_k_no_ties



2023-06-29 11:00:29.485251: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 11:00:29.487284: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 11:00:29.531060: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 11:00:29.531630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 11:00:30.794804: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
data_path='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/clean_annual_tract/'


In [3]:
best_model = 'n0.3_samp50_lr0.01_hs50_10_lb5'
noise = 0.3
perturbation_samples = 50
hidden_sizes = [50, 10]
lookback_years=5
learning_rate=0.01
add_spacetime=True
add_svi=True

In [5]:
epochs = 3000
seed = 360
time_window = lookback_years
timesteps_per_year = 1
first_train_eval_year = 2013
last_train_eval_year = 2017
batch_dim_size = (last_train_eval_year - first_train_eval_year + 1)*timesteps_per_year
validation_year = 2018
first_test_year = 2019
last_test_year = 2020


tf.random.set_seed(seed)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['deaths']

if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


data_gdf = gpd.read_file(data_path)

multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

train_x_BSF_flat, train_y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=timesteps_per_year)

valid_x_BSF_flat, valid_y_BS = make_data(multiindexed_gdf, validation_year, validation_year,
                                         time_window, features_only, train_shape, pred_lag=timesteps_per_year)

test_x_BSF_flat, test_y_BS = make_data(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=timesteps_per_year)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)

2023-06-29 11:06:17.562253: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-29 11:06:17.562295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2023-06-29 11:06:17.562303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: s1cmp008.pax.tufts.edu
2023-06-29 11:06:17.562424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 515.65.1
2023-06-29 11:06:17.562456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 515.65.1
2023-06-29 11:06:17.562462: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 515.65.1


In [7]:
true_deaths = multiindexed_gdf.loc[idx[:,18],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
deaths_index = true_deaths.index.values
num_tracts = len(deaths_index)
removed_tracts = 250
num_sampled = num_tracts - removed_tracts
K = 100
rng = np.random.default_rng(seed=360)

ratios = []

for _ in range(5000):

    sampled_indicies = rng.choice(deaths_index, size=num_sampled, replace=False)



    top_K_orig_deaths = true_deaths.sort_values(ascending=False)[:K].sum()
    top_K_sampled_deaths = true_deaths[sampled_indicies].sort_values(ascending=False)[:K].sum()
    ratio = top_K_sampled_deaths/top_K_orig_deaths
    ratios.append(ratio)

print(f'Removed: {removed_tracts}, {np.mean(ratios):2f}')


Removed: 250, 0.949377


In [ ]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2018)&(multiindexed_gdf['year']>2017)].groupby(level='geoid')['deaths'].mean()
test_time = 19
for _ in range(5000):
    this_2019 = multiindexed_gdf.loc[idx[:,test_time],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    sampled_indicies = rng.choice(deaths_index, size=num_sampled, replace=False)
    
    bprs.append(fixed_top_X(this_2019[sampled_indicies],
                            avg_2018[sampled_indicies],
                            X=K)[-1])
avg_2019 = multiindexed_gdf[(multiindexed_gdf['year']<=2019)&(multiindexed_gdf['year']>2018)].groupby(level='geoid')['deaths'].mean()
test_time = 20
for _ in range(5000):
    this_2020 = multiindexed_gdf.loc[idx[:,test_time],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    
    bprs.append(fixed_top_X(this_2020,avg_2019,X=100)[-1])
np.mean(bprs)